In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
%matplotlib inline

In [11]:
train= pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
test["playtime_forever"]=0
data = pd.concat([train,test],axis=0, sort=False)
data.fillna(0, inplace=True)

In [12]:
del data['id']
del data['is_free']


dummy1 = data["genres"].str.get_dummies(",")
del data["genres"]

dummy2 = data["categories"].str.get_dummies(",")
del data["categories"]

dummy3=data["tags"].str.get_dummies(",")
test_dummy3=test["tags"].str.get_dummies(",")
del data["tags"]

for var in dummy1.columns:
    if var in dummy2.columns:
        del dummy1[var]
    if var in dummy3.columns:
        del dummy1[var]
for var in dummy2.columns:
    if var in dummy3.columns:
        del dummy2[var]

In [13]:
# operate datetime variable
a=pd.to_datetime(data["purchase_date"])
b=pd.datetime(2019,11,1)-a
#b=a-pd.datetime(2009,11,1)
data["purchase_date"]=b.astype('timedelta64[D]')
#A=pd.to_datetime(test_merge["purchase_date"])
#B=pd.datetime(2019,11,1)-A
#test_merge["purchase_date"]=B.astype('timedelta64[D]')

a=pd.to_datetime(data["release_date"])
b=pd.datetime(2019,11,1)-a
#b=a-pd.datetime(2009,11,1)
data["release_date"]=b.astype('timedelta64[D]')
#A=pd.to_datetime(test_merge["release_date"])
#B=pd.datetime(2019,11,1)-A
#test_merge["release_date"]=b.astype('timedelta64[D]')
np.where(np.isnan(data))

(array([], dtype=int64), array([], dtype=int64))

In [14]:
y=data["playtime_forever"].values
del data["playtime_forever"]

In [15]:
array_data=data.values
array_tag=dummy1.values
array_tag2=dummy2.values
array_tag3=dummy3.values
x=np.hstack([array_data,array_tag,array_tag2,array_tag3])
x_train,x_test=x[:357],x[357:]
y_train,y_test=y[:357],y[357:]
y_train=y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)

In [16]:
#x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,train_size=0.9)
std_x = MinMaxScaler() 
x_train = std_x.fit_transform(x_train)
#x_val = std_x.transform(x_val)
x_test = std_x.transform(x_test)
 
std_y  = MinMaxScaler() 
y_train = std_y.fit_transform(y_train)
#y_val = std_y.transform(y_val)

In [17]:
model=keras.Sequential()
#model.add(keras.layers.Dense(8,))
model.add(keras.layers.Dense(4, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
model.add(keras.layers.Dense(2, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
#model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1))


model.compile(optimizer=tf.train.AdamOptimizer(0.003),
              loss='mse',metrics=['mse'])

model.fit(x_train, y_train, epochs=100, batch_size=35)

Epoch 1/100
357/357 [==============================] - 0s 294us/sample - loss: 0.0261 - mean_squared_error: 0.0157
Epoch 2/100
357/357 [==============================] - 0s 64us/sample - loss: 0.0195 - mean_squared_error: 0.0111
Epoch 3/100
357/357 [==============================] - 0s 84us/sample - loss: 0.0168 - mean_squared_error: 0.0098
Epoch 4/100
357/357 [==============================] - 0s 64us/sample - loss: 0.0146 - mean_squared_error: 0.0084
Epoch 5/100
357/357 [==============================] - 0s 76us/sample - loss: 0.0128 - mean_squared_error: 0.0073
Epoch 6/100
357/357 [==============================] - 0s 73us/sample - loss: 0.0116 - mean_squared_error: 0.0065
Epoch 7/100
357/357 [==============================] - 0s 64us/sample - loss: 0.0105 - mean_squared_error: 0.0057
Epoch 8/100
357/357 [==============================] - 0s 81us/sample - loss: 0.0095 - mean_squared_error: 0.0051
Epoch 9/100
357/357 [==============================] - 0s 67us/sample - loss: 0.0086 - 

357/357 [==============================] - 0s 73us/sample - loss: 0.0028 - mean_squared_error: 0.0014
Epoch 73/100
357/357 [==============================] - 0s 95us/sample - loss: 0.0029 - mean_squared_error: 0.0015
Epoch 74/100
357/357 [==============================] - 0s 62us/sample - loss: 0.0029 - mean_squared_error: 0.0016
Epoch 75/100
357/357 [==============================] - 0s 70us/sample - loss: 0.0028 - mean_squared_error: 0.0015
Epoch 76/100
357/357 [==============================] - 0s 64us/sample - loss: 0.0028 - mean_squared_error: 0.0015
Epoch 77/100
357/357 [==============================] - 0s 62us/sample - loss: 0.0028 - mean_squared_error: 0.0015
Epoch 78/100
357/357 [==============================] - 0s 59us/sample - loss: 0.0027 - mean_squared_error: 0.0014
Epoch 79/100
357/357 [==============================] - 0s 62us/sample - loss: 0.0026 - mean_squared_error: 0.0014
Epoch 80/100
357/357 [==============================] - 0s 73us/sample - loss: 0.0026 - mean_

In [18]:
final_predict=model.predict(x_test)
y_final_predict = std_y.inverse_transform(final_predict)
y_final_predict=(y_final_predict>0)*y_final_predict
b=np.arange(90).reshape(90,1)
c=np.c_[b,y_final_predict]
np.savetxt('predict.csv', c, delimiter = ',')